In [177]:
import pandas as pd
import skbio
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa, pcoa_biplot
import altair as alt

In [178]:
def visualize_abundance(df, df_tax, level="tax_id"):
    full_tax_columns = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'tax_id']
    
    # Remove the specified level from tax_columns
    tax_columns = [col for col in full_tax_columns if col != level]
    
    df = df.drop(columns=tax_columns)
    
    # Set tax_id as the index
    df.set_index(level, inplace=True)
    
    # Calculate total abundance for each tax_id by summing across the rows
    total_abundance = df.sum(axis=1)
    
    # Identify the top 10 tax_id by total abundance
    top_10_tax_ids = total_abundance.nlargest(10).index
    
    # Define colors for the top 10 tax_id
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {tax_id: color for tax_id, color in zip(top_10_tax_ids, colors)}
    
    # Assign grey color for other tax_id
    color_map['other'] = 'grey'
    
    # Add a color column to the DataFrame
    df['color'] = df.index.to_series().apply(lambda x: color_map[x] if x in color_map else color_map['other'])
    
    # Reset the index to melt the DataFrame
    df.reset_index(inplace=True)
    
    # Melt the DataFrame to long format for Altair
    df_melted = df.melt(id_vars=[level, 'color'], var_name='barcode', value_name='relative_abundance')
    
    # Drop rows with NaN values in 'relative_abundance'
    df_melted = df_melted.dropna(subset=['relative_abundance'])
    # Enrich the melted DataFrame with taxonomic information using a lambda function
    for col in tax_columns:
        df_melted[col] = df_melted[level].apply(lambda x: df_tax.at[int(x), col] if x != "unassigned" and int(x) in df_tax.index else None)
    
    # Create the Altair chart
    chart = alt.Chart(df_melted).mark_bar().encode(
        x='barcode:N',
        y='relative_abundance:Q',
        color=alt.Color('color:N', scale=None),
        tooltip=full_tax_columns + ['relative_abundance']
    ).properties(
        title='Relative Distribution of Taxa ID'
    )
    
    # Display the chart
    chart.show()

In [179]:
df = pd.read_csv("../data/processed/emu/emu-combined-tax_id.tsv", sep="\t")
df_tax = pd.read_csv("../resources/database/silva/taxonomy_split.tsv", sep="\t", index_col=0)
tax_columns = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for col in tax_columns:
    df[col] = df['tax_id'].apply(lambda x: df_tax.at[int(x), col] if x != "unassigned" and int(x) in df_tax.index else None)

In [ ]:
visualize_abundance(df, df_tax, level="tax_id")

In [ ]:
df

In [182]:
def convert_to_gtdb_taxonomy(row, max_level='species'):
    taxonomy_levels = [
        ('superkingdom', 'd__'),
        ('phylum', 'p__'),
        ('class', 'c__'),
        ('order', 'o__'),
        ('family', 'f__'),
        ('genus', 'g__'),
        ('species', 's__')
    ]
    
    # Create a dictionary to map levels to their order
    level_order = {level: i for i, (level, _) in enumerate(taxonomy_levels)}
    
    # Filter the taxonomy levels up to the specified max_level
    filtered_levels = [(level, prefix) for level, prefix in taxonomy_levels if level_order[level] <= level_order[max_level]]
    
    taxonomy_string = ",".join([f"{prefix}{row[level]}" for level, prefix in filtered_levels if pd.notna(row[level])])
    return taxonomy_string

def visualize_abundance(df_matrices, max_level='species', n_abundance=10, width=300, height=300):
    df_matrices = df_matrices.copy()
    # Calculate total abundance for each tax_id by summing across the rows
    total_abundance = df_matrices.sum(axis=1)
    
    # Identify the top 10 tax_id by total abundance
    top_10_tax_ids = total_abundance.nlargest(n_abundance).index
    
    # Define colors for the top 10 tax_id
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {tax_id: color for tax_id, color in zip(top_10_tax_ids, colors)}
    
    # Assign grey color for other tax_id
    color_map['other'] = 'grey'
    
    # Add a color column to the DataFrame
    df_matrices['color'] = df_matrices.index.to_series().apply(lambda x: color_map[x] if x in color_map else color_map['other'])
    
    # Reset the index to melt the DataFrame
    df_matrices.reset_index(inplace=True)
    
    # Melt the DataFrame to long format for Altair
    df_melted = df_matrices.melt(id_vars=['taxonomy', 'color'], var_name='barcode', value_name='relative_abundance')
    
    # Drop rows with NaN values in 'relative_abundance'
    df_melted = df_melted.dropna(subset=['relative_abundance'])
    
    # Create the Altair chart
    chart = alt.Chart(df_melted).mark_bar().encode(
        x='barcode:N',
        y='relative_abundance:Q',
        color=alt.Color('color:N', scale=None),
        tooltip=['taxonomy', 'relative_abundance']
    ).properties(
        title=f'Relative Distribution at {max_level} level',
        width=width,
        height=height
    )
    
    return chart

def plot_pcoa_with_biplot(df_matrix, N=5, max_level=5, width=300, height=300):
    """
    Perform PCoA, compute the biplot projection, and visualize the results with vectors for the most significant contributor taxa.

    Parameters:
    df_matrix (pd.DataFrame): DataFrame containing the data matrix (samples by features).
    N (int): Number of top taxa to display.

    Returns:
    alt.Chart: Altair chart object with the PCoA plot and biplot vectors.
    """
    # Calculate the Bray-Curtis distance matrix
    df_matrix = df_matrix.fillna(0)
    distance_matrix = beta_diversity('braycurtis', df_matrix.T, ids=df_matrix.columns)

    # Perform PCoA
    pcoa_results = pcoa(distance_matrix)

    # Compute the biplot projection
    biplot_results = pcoa_biplot(pcoa_results, df_matrix.T)

    # Create a DataFrame for the PCoA results
    pcoa_df = pcoa_results.samples.reset_index()

    # Create a DataFrame for the biplot scores
    biplot_df = biplot_results.features

    # Select the top N taxa based on their contribution to the variability
    top_taxa = biplot_df.abs().sum(axis=1).nlargest(N).index
    top_taxa_scores = biplot_df.loc[top_taxa]

    # Create a DataFrame for the vectors
    vectors_df = pd.DataFrame({
        'PC1_start': [0] * len(top_taxa_scores),
        'PC2_start': [0] * len(top_taxa_scores),
        'PC1_end': top_taxa_scores['PC1'],
        'PC2_end': top_taxa_scores['PC2'],
        'taxa': top_taxa_scores.index
    })

    # Plot the PCoA results using Altair
    base = alt.Chart(pcoa_df).mark_circle(size=100).encode(
        x=alt.X('PC1', title=f'PC1 ({pcoa_results.proportion_explained.iloc[0]:.2%} variance)'),
        y=alt.Y('PC2', title=f'PC2 ({pcoa_results.proportion_explained.iloc[1]:.2%} variance)'),
        tooltip=['index', 'PC1', 'PC2']
    ).properties(
        title=f'PCoA at {max_level} level',
        width=600,
        height=600
    )

    # Vertical line at x=0
    vline = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(color='red').encode(
        x='x:Q'
    )

    # Horizontal line at y=0
    hline = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='red').encode(
        y='y:Q'
    )

    # Create arrows for the top taxa
    arrows = alt.Chart(vectors_df).mark_line(color='blue').encode(
        x='PC1_start:Q',
        x2='PC1_end:Q',
        y='PC2_start:Q',
        y2='PC2_end:Q',
        tooltip=['taxa']
    )

    # Add text labels for the vectors
    vector_labels = alt.Chart(vectors_df).mark_text(
        align='left',
        baseline='middle',
        dx=5,
        dy=-5,
        color='blue'
    ).encode(
        x='PC1_end:Q',
        y='PC2_end:Q',
        text='taxa'
    )

    # Combine the base chart with the lines and vectors
    chart = base + vline + hline + arrows + vector_labels
    chart = chart.properties(
        width=width,
        height=height
    )

    return chart

In [ ]:

# Display the DataFrame with the new column
dropped_columns = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']

# Apply the function to every row and create a new column
max_levels = ['phylum', 'class', 'order', 'family', 'genus', 'species']  # Change this to the desired level
matrices = []
charts = []
biplots = []
for max_level in max_levels:
    df['taxonomy'] = df.apply(convert_to_gtdb_taxonomy, axis=1, max_level=max_level)
    df_matrix = df.copy().drop(columns=['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species',]).set_index("tax_id").fillna(0)
    df_matrix.loc["unassigned", "taxonomy"] = "unassigned"
    df_matrix = df_matrix.set_index("taxonomy")
    df_matrix.index = df_matrix.index.map(lambda x: x.split(",")[-1])
    df_matrix = df_matrix.groupby(df_matrix.index).sum()
    matrices.append(df_matrix)
    charts.append(visualize_abundance(df_matrix, max_level=max_level, n_abundance=5))
    biplots.append(plot_pcoa_with_biplot(df_matrix, max_level=max_level, N=5))


In [ ]:
(charts[0] | charts[1] | charts[2]) & (charts[3] | charts[4] | charts[5])

In [ ]:
(biplots[0] | biplots[1] | biplots[2]) & (biplots[3] | biplots[4] | biplots[5])